In [75]:
import numpy as np
import pandas as pd
import requests


In [76]:
# Getting token and store (Valid for 1 day)
headers = {"accesskey":"f84b64c0-d851-4282-8aa2-da0e1bd4bde9",
'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "en-US,en;q=0.5","Accept-Encoding": "gzip, deflate"}

response = requests.get("https://www.ura.gov.sg/uraDataService/insertNewToken.action", headers=headers)
token = response.json()['Result']
print(token)

# Getting carpark list 
headers = {"accesskey":"f84b64c0-d851-4282-8aa2-da0e1bd4bde9", "token":token,
'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "en-US,en;q=0.5","Accept-Encoding": "gzip, deflate"}

response = requests.get("https://www.ura.gov.sg/uraDataService/invokeUraDS?service=Car_Park_Details", headers=headers)
carparklist = response.json()
# print(carparklist['Result'])

carpark_df =  pd.DataFrame(carparklist['Result']).drop_duplicates(subset='ppCode', keep="last")

carpark_df = carpark_df.rename(columns={'ppCode': 'carparkNo'})
carpark_df = carpark_df.drop(columns=['weekdayMin', 'weekdayRate', 'parkingSystem', 'ppName', 'vehCat', 'satdayMin'
                                      ,'satdayRate','sunPHMin', 'sunPHRate', 'startTime', 'parkCapacity', 'endTime', 'remarks'])
carpark_df = carpark_df.set_index('carparkNo')

print(len(carpark_df))

# Dropping carpark with no coordinates return
carpark_df = carpark_df[carpark_df['geometries'].map(len) > 0]

print(len(carpark_df))
print(carpark_df.head())



8e3T85-9Jde19HWab66Q2nS1XJ1A4Y2C1MN7a6ednv6-adcZ09Aa1k105Sd1v2Ru1HF1x1bn68x2gzE-7uZc2-V-dndGdQYbe05a
764
762
                                           geometries
carparkNo                                            
A0004       [{'coordinates': '31063.017,31665.0933'}]
A0005      [{'coordinates': '30873.0177,32533.6617'}]
A0006        [{'coordinates': '29548.345,29292.548'}]
A0007       [{'coordinates': '27582.895,31962.0601'}]
A0008      [{'coordinates': '29394.9904,29258.8791'}]


In [77]:
# Splitting the geometries and get the lat and lng out as separate column for greater ease in analysis

geometry = carpark_df['geometries']
# print(geometry)

coordinates = geometry.str[0]
# print(coordinates)

xAndy = coordinates.str['coordinates'].str.split(",")
# print(lngAndLat)

x = xAndy.str[0]
y = xAndy.str[1]


# # Add Longitutde and Latitude back to main dataframe
carpark_df["x"] = x
carpark_df["y"] = y

# print(carpark_df)
latList = []
lngList = []


for lng,lat in zip(carpark_df.x, carpark_df.y):
    print(lng,lat)
    response = requests.get("https://developers.onemap.sg/commonapi/convert/3414to4326?X=" + str(lng) + "&Y=" + str(lat))
    print(response.json())
    latList.append(response.json()['latitude'])
    lngList.append(response.json()['longitude'])


print(latList)
print(lngList)


carpark_df["latitude"] = latList
carpark_df["longitude"] = lngList

print(carpark_df.head())



31063.017 31665.0933
{'latitude': 1.302642219937201, 'longitude': 103.86084119330316}
30873.0177 32533.6617
{'latitude': 1.3104972637937171, 'longitude': 103.85913404261238}
29548.345 29292.548
{'latitude': 1.2811858670982985, 'longitude': 103.84723105375262}
27582.895 31962.0601
{'latitude': 1.3053280313222764, 'longitude': 103.82957069514482}
29394.9904 29258.8791
{'latitude': 1.2808813845277496, 'longitude': 103.84585310232585}
30770.0173 31622.8216
{'latitude': 1.3022599571413955, 'longitude': 103.85820845289837}
29730.2995 30701.2921
{'latitude': 1.2939260442330984, 'longitude': 103.84886606187924}
30638.3394 31698.2199
{'latitude': 1.3029418442262124, 'longitude': 103.85702527311646}
29542.3676 29248.8218
{'latitude': 1.2807904229808036, 'longitude': 103.84717734239695}
25838.8021 34044.9773
{'latitude': 1.3241651253008084, 'longitude': 103.81389905715548}
25863.4915 34079.5138
{'latitude': 1.3244774629124667, 'longitude': 103.8141209022391}
33156.0403 34185.9516
{'latitude': 1.3

In [78]:
# Removing unused column

carpark_df = carpark_df.drop('geometries', axis=1)
carpark_df = carpark_df.drop('x', axis=1)
carpark_df = carpark_df.drop('y', axis=1)

In [79]:
# Calculating distance to CBD (Raffles place mrt as the coordinate)
from geopy.distance import geodesic

def compute_distance(address_row):
    address_lat, address_lng = address_row[['latitude','longitude']]
    distance = geodesic((address_lat, address_lng), (1.283931, 103.851461)).km
    return distance

distance_series = carpark_df.apply(compute_distance, axis=1)

carpark_df["distance to cbd (km)"] = distance_series

print(carpark_df.head())

           latitude   longitude  distance to cbd (km)
carparkNo                                            
A0004      1.302642  103.860841              2.317438
A0005      1.310497  103.859134              3.059163
A0006      1.281186  103.847231              0.560135
A0007      1.305328  103.829571              3.396012
A0008      1.280881  103.845853              0.709386


In [80]:
# Calculating distance to nearest shopping mall

df_malls = pd.read_csv('shopping_mall_coordinates.csv')

def compute_distance(address_row, df_malls):
    address_lat, address_lng = address_row[['latitude','longitude']]
    min_distance = 9999.0
    mall = ""
    for mall_lat, mall_lng in zip(df_malls['LATITUDE'], df_malls['LONGITUDE']):
        distance = geodesic((address_lat, address_lng), (mall_lat, mall_lng)).km
        if distance < min_distance:
            min_distance = distance

    return distance

distance_series = carpark_df.apply(compute_distance, axis=1, df_malls=df_malls)
carpark_df["distance to nearest shopping mall (km)"] = distance_series

print(carpark_df.head())

           latitude   longitude  distance to cbd (km)  \
carparkNo                                               
A0004      1.302642  103.860841              2.317438   
A0005      1.310497  103.859134              3.059163   
A0006      1.281186  103.847231              0.560135   
A0007      1.305328  103.829571              3.396012   
A0008      1.280881  103.845853              0.709386   

           distance to nearest shopping mall (km)  
carparkNo                                          
A0004                                    6.605997  
A0005                                    6.699703  
A0006                                    4.930585  
A0007                                    3.510513  
A0008                                    4.784012  


In [81]:
# Calculating distance to nearest university

df_university = pd.read_csv('UniversitySingapore.csv')

def compute_distance(address_row, df_university):
    address_lat, address_lng = address_row[['latitude','longitude']]
    min_distance = 9999.0
    for university_lat, university_lng in zip(df_university['Latitude'], df_university['Longitude']):
        distance = geodesic((address_lat, address_lng), (university_lat, university_lng)).km
        if distance < min_distance:
            min_distance = distance

    return min_distance

distance_series = carpark_df.apply(compute_distance, axis=1, df_university=df_university)
carpark_df["distance to nearest university (km)"] = distance_series

print(carpark_df.head())

           latitude   longitude  distance to cbd (km)  \
carparkNo                                               
A0004      1.302642  103.860841              2.317438   
A0005      1.310497  103.859134              3.059163   
A0006      1.281186  103.847231              0.560135   
A0007      1.305328  103.829571              3.396012   
A0008      1.280881  103.845853              0.709386   

           distance to nearest shopping mall (km)  \
carparkNo                                           
A0004                                    6.605997   
A0005                                    6.699703   
A0006                                    4.930585   
A0007                                    3.510513   
A0008                                    4.784012   

           distance to nearest university (km)  
carparkNo                                       
A0004                                 0.433330  
A0005                                 0.979376  
A0006                           

In [82]:
# Calculating distance to nearest pre-school

df_school = pd.read_csv('preschool_information.csv')

def compute_distance(address_row, df_school):
    address_lat, address_lng = address_row[['latitude','longitude']]
    min_distance = 9999.0
    for school_lat, school_lng in zip(df_school['Latitude'], df_school['Longitude']):
        distance = geodesic((address_lat, address_lng), (school_lat, school_lng)).km
        if distance < min_distance:
            min_distance = distance
    return min_distance

distance_series = carpark_df.apply(compute_distance, axis=1, df_school=df_school)
carpark_df["distance to nearest pre-school (km)"] = distance_series

print(carpark_df.head())

           latitude   longitude  distance to cbd (km)  \
carparkNo                                               
A0004      1.302642  103.860841              2.317438   
A0005      1.310497  103.859134              3.059163   
A0006      1.281186  103.847231              0.560135   
A0007      1.305328  103.829571              3.396012   
A0008      1.280881  103.845853              0.709386   

           distance to nearest shopping mall (km)  \
carparkNo                                           
A0004                                    6.605997   
A0005                                    6.699703   
A0006                                    4.930585   
A0007                                    3.510513   
A0008                                    4.784012   

           distance to nearest university (km)  \
carparkNo                                        
A0004                                 0.433330   
A0005                                 0.979376   
A0006                       

In [83]:
# Calculating distance to nearest primary school

df_school = pd.read_csv('primaryschoolsg.csv')

def compute_distance(address_row, df_school):
    address_lat, address_lng = address_row[['latitude','longitude']]
    min_distance = 9999.0
    for school_lat, school_lng in zip(df_school['Latitude'], df_school['Longitude']):
        distance = geodesic((address_lat, address_lng), (school_lat, school_lng)).km
        if distance < min_distance:
            min_distance = distance
    return min_distance

distance_series = carpark_df.apply(compute_distance, axis=1, df_school=df_school)
carpark_df["distance to nearest primary school (km)"] = distance_series

print(carpark_df.head())

           latitude   longitude  distance to cbd (km)  \
carparkNo                                               
A0004      1.302642  103.860841              2.317438   
A0005      1.310497  103.859134              3.059163   
A0006      1.281186  103.847231              0.560135   
A0007      1.305328  103.829571              3.396012   
A0008      1.280881  103.845853              0.709386   

           distance to nearest shopping mall (km)  \
carparkNo                                           
A0004                                    6.605997   
A0005                                    6.699703   
A0006                                    4.930585   
A0007                                    3.510513   
A0008                                    4.784012   

           distance to nearest university (km)  \
carparkNo                                        
A0004                                 0.433330   
A0005                                 0.979376   
A0006                       

In [84]:
# Calculating distance to nearest secondary school

df_school = pd.read_csv('secondaryschoolsg.csv')

def compute_distance(address_row, df_school):
    address_lat, address_lng = address_row[['latitude','longitude']]
    min_distance = 9999.0
    for school_lat, school_lng in zip(df_school['Latitude'], df_school['Longitude']):
        distance = geodesic((address_lat, address_lng), (school_lat, school_lng)).km
        if distance < min_distance:
            min_distance = distance
    return min_distance

distance_series = carpark_df.apply(compute_distance, axis=1, df_school=df_school)
carpark_df["distance to nearest secondary school (km)"] = distance_series

print(carpark_df.head())

           latitude   longitude  distance to cbd (km)  \
carparkNo                                               
A0004      1.302642  103.860841              2.317438   
A0005      1.310497  103.859134              3.059163   
A0006      1.281186  103.847231              0.560135   
A0007      1.305328  103.829571              3.396012   
A0008      1.280881  103.845853              0.709386   

           distance to nearest shopping mall (km)  \
carparkNo                                           
A0004                                    6.605997   
A0005                                    6.699703   
A0006                                    4.930585   
A0007                                    3.510513   
A0008                                    4.784012   

           distance to nearest university (km)  \
carparkNo                                        
A0004                                 0.433330   
A0005                                 0.979376   
A0006                       

In [85]:
carpark_df.to_csv('carparklist_master_Information.csv')